### <font color='#4285f4'>Overview</font>

This process streamlines marketing video creation by having Gemini simultaneously generate both the script and corresponding Veo 2 prompts, incorporating ABCD best practices for each 6-second segment. Veo 2 then generates the video segments, which are narrated with English and French voice overs using Vertex AI's text-to-speech. Finally, Gemini performs a quality assurance check on the assembled video.

Process Flow:

1. Transform the Veo 2 comprehensive documentation into a set of clear instructions that specifically guide the creation of effective Veo 2 prompts.
2. Create a prompt for Gemini that will:
    * a. A prompt should include "styles", "composition", "Ambiance & Emotions", "Cinematic effects" (doc)
    * a. Let Gemini know that the video will be generated in 6 second segments
    * b. Ask Gemini for the script and the video prompt
    * c. Also, inform Gemini about the ABCD detector rules so it can determine which rules apply per segment
3. Use GenAI to generate videos via Veo 2
4. Use Vertex AI text-to-speech to generate the voice over using a British accent and a male voice
5. Use Vertex AI text-to-speech to generate the voice over using a French accent and a male voice
6. Overlay the text-to-speech to each video segment
7. Merge all the video segments into one final video.
8. Ask Gemini for a Quality Assurance check of the generated video.

Notes:

1. The notebook does merge the videos, but the quality is much better when you merge with a high quality video tool. The text-to-speech does not always align perfectly with the video.
2. You can also adjust the speed of the text-to-speech to control how fast the words are spoken.

Cost:
* Veo 2: 50 cents per second of video 
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author: 
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Artifacts/Campaign-Assets-Text-to-Video-02-Architecture.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[![Video](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/adam-paternostro-video.png)](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Assets-Text-to-Video-02.mp4)


In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Assets-Text-to-Video-02.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

### <font color='#4285f4'>Pip installs</font>

In [ ]:
# PIP Installs
import sys

# https://pypi.org/project/moviepy/
!{sys.executable} -m pip install moviepy

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "${bigquery_location}"
region = "${region}"
location = "${location}"
storage_account = "${chocolate_ai_bucket}"
video_number_of_segments = 10
video_segment_length = 6
video_length_in_seconds = video_number_of_segments * video_segment_length
public_storage_storage_account = "data-analytics-golden-demo"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RetryCondition (for retrying LLM calls)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-2.0-flash", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-2.0-flash", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{multimodal_prompt_list[0]}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Text to Speech Generation

In [ ]:
def TextToSpeechLanguageList(language_code):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/voices/list
  url = f"https://texttospeech.googleapis.com/v1/voices?languageCode={language_code}"

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    return response.text
  else:
    error = f"Error with language_code:'{language_code}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

In [ ]:
def TextToSpeech(local_filename, text, language_code, language_code_name, ssml_gender, speaking_rate = 1):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/text/synthesize
  url = f"https://texttospeech.googleapis.com/v1/text:synthesize"

  payload = {
   "input": {
      "text": text
   },
   "voice": {
      "languageCode": language_code,
      "name": language_code_name,
      "ssmlGender": ssml_gender # FEMALE | MALE
   },
   "audioConfig": {
      "audioEncoding": "MP3",
      "speakingRate": speaking_rate,
   }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    audio_data = json.loads(response.content)["audioContent"]
    audio_data = base64.b64decode(audio_data)
    with open(local_filename, "wb") as f:
      f.write(audio_data)
    print(f"Audio generated OK.")
    return local_filename
  else:
    error = f"Error with text:'{text}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Overlay text to speech to video

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

def MergeVideoAndAudio(video_filename, audio_filename, output_filename):
  # Load the video and audio files
  video = VideoFileClip(video_filename)
  audio = AudioFileClip(audio_filename)

  # Combine the video and audio
  final_clip = video.set_audio(audio)

  # Save the combined video
  final_clip.write_videofile(output_filename)

#### Combine Videos

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def merge_videos_sorted(folder_path, output_video_name):
  """
  Merges all MP4 video files in the specified folder into a single video,
  sorted by file name.

  Args:
      folder_path: The path to the folder containing the videos.
  """

  video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]
  video_files.sort()  # Sort the files by name

  clips = [VideoFileClip(os.path.join(folder_path, video)) for video in video_files]

  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile(os.path.join(folder_path, output_video_name))

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

In [ ]:
def download_from_gcs(destination_file_name, gcs_storage_bucket, object_name):
  # prompt: Write python code to download a blob from a gcs bucket.  do not use the requests method

  from google.cloud import storage
  storage_client = storage.Client()
  bucket = storage_client.bucket(gcs_storage_bucket)

  # Construct a client side representation of a blob.
  # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
  # any content from Google Cloud Storage. As we don't need additional data,
  # using `Bucket.blob` is preferred here.
  blob = bucket.blob(object_name)
  blob.download_to_filename(destination_file_name)

  print(
      "Downloaded storage object {} from bucket {} to local file {}.".format(
          object_name, gcs_storage_bucket, destination_file_name
      )
  )

In [ ]:
# prompt: python to delete a file even if it does not exist

def delete_file(filename):
  try:
    os.remove(filename)
    print(f"File '{filename}' deleted successfully.")
  except FileNotFoundError:
    print(f"File '{filename}' not found.")

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

### <font color='#4285f4'>Brainstorming with Gemini</font>

##### Sample Prompt

```
You are a marketing expert and are creating a marketing video for a company that sells premium handcrafted chocolates, handmade desserts and delicious coffee drinks.
Write a 60-second video script that showcases the artistry of chocolatiers crafting exquisite desserts.
This is a new company Chocolate AI who wants to build their brand awarness.
The company is based in Paris, France.
The video will be generated by GenAI and needs to be magicical and spark the imagination of the viewers.
Feature a range of signature offerings from the below menu.
Encourage unconventional ideas and fresh perspectives and inspires unique variations when viewing the video.
The video should be shot inside the cholcolate shop or show the street outside the shop.

Output Fields:
- "marketing-campaign-overview",
- "segments"
  - "segement-number": The segment number.
  - "segement-numbervisuals": The segment number.
  - "visuals": The visual screen that is being described.
  - "voiceover"  The script for the text-to-speech.  The number of words should be about 6 seconds when spoken.
  - "text-to-video-prompt":
    - Read the  "Text-to-Video Prompt Writing Help" to learn more about how to create good text-to-video prompts.
    - Make sure you include all the relevant best practices when creating the text-to-video prompt:
    - A detailed prompt for generating the video using text-to-video technology.
    - Focus on creating the menu item with an artistic flair.
    - Do not include "text overlays" in the text-to-video prompt.
    - The prompt should also reference that we are in a chocolate, dessert, coffee shop in Paris so it knows the context of the video.
    - Do not include children in the text-to-video prompt.
  - "ABCD-rules": The relevant ABCD rules that should be applied to the segment.  The ABCD rules are a specific set of guidelines that you may or may not be using.


Text-to-Video Prompt Writing Help:
<text-to-video-prompt-guide>
Here are some our best practices for text-to-video prompts:

Detailed prompts = better videos:
  - More details you add, the more control you’ll have over the video. A prompt should look like this:
  - Example Prompt: "Camera dollies to show a close up of a desperate man in a green trench coat is making a call on a rotary style wall-phone, green neon light, movie scene."
    - Here is a break down of a text-to-video prompt:
    - Camera dollies to show = Camera motion
    - A close up of = Composition
    - A desperate man in a green trench coat = Subject
    - Is making a call = Action
    - On a roary style wall-phone = Scene
    - Green Neon light = Ambiance
    - Movie Scene = Style

Use the right keywords for better control:
  - We’ve identified a list of some keywords that work well with text-to-video, use these in your human written prompts to get the desired camera motion or style.
  - Subject: Who or what is the main focus of the shot e.g. happy woman in her 30s
  - Scene: Where is the location of the shot (on a busy street, in space)
  - Action: What is the subject doing (walking, running, turning head)
  - Camera Motion: What the camera is doing e.g. POV shot, Aerial View, Tracking Drone view, Tracking Shot
    - Example Camera Motion: "Tracking drone view of a man driving a red convertible car in Palm Springs, 1970s, warm sunlight, long shadows"
    - Example Camera Motion: "A POV shot from a vintage car driving in the rain, Canada at night, cinematic"


Styles:
   - Overall aesthetic. Consider using specific film style keywords e.g. horror film, film noir or animated styles e.g. 3D cartoon style render
  - Example Prompt: "Over the shoulder of a young woman in a car, 1970s, film grain, horror film, cinematic he Film noir style, man and woman walk on the street, mystery, cinematic, black and white"
  - Example Prompt: "A cute creatures with snow leopard-like fur is walking in winter forest, 3D cartoon style render. An architectural rendering of a white concrete apartment building with flowing organic shapes, seamlessly blending with lush greenery and futuristic elements."

Composition:
  - How the shot is framed. This is often relative to the subject e.g. wide shot, close-up, low angle
  - Example Prompt: "Extreme close-up of a an eye with city reflected in it. A wide shot of surfer walking on a beach with a surfboard, beautiful sunset, cinematic"

Ambiance & Emotions:
  - How the color and light contribute to the scene (blue tones, night)
  - Example Prompt: "A close-up of a girl holding adorable golden retriever puppy in the park, sunlight Cinematic close-up shot of a sad woman riding a bus in the rain, cool blue tones, sad mood"

Cinematic effects:
  - e.g. double exposure, projected, glitch camera effect
  - Example Prompt: "A double exposure of silhouetted profile of a woman walking and lake, walking in a forest Close-up shot of a model with blue light with geometric shapes projected on her face"
  - Example Prompt: "Silhouette of a man walking in collage of cityscapes Glitch camera effect, close up of woman’s face speaking, neon colors"
</text-to-video-prompt-guide>


<ai-sweets-menu>
[{"menu_description":"A decadent blend of rich dark chocolate infused with aromatic lavender, topped with a sprinkle of delicate sea salt.","menu_name":"Lavender Sea Salt Chocolate"},{"menu_description":"A blend of dark and white chocolate formed into a hollow egg. Filled with a variety or truffles.","menu_name":"Parisian Chocolate Easter Egg"},{"menu_description":"A triple chocolate layered cake, with a milk chocolate ganache, topped with dark chocolate shavings.","menu_name":"Triple Chocolate Cake"},{"menu_description":"Three layers of decadent chocolate cake, with alternating layers of rich dark chocolate ganache and a smooth milk chocolate mousse.","menu_name":"Triple Chocolate Symphony Cake"},{"menu_description":"Five layers of decadent chocolate cake, each infused with a different exotic spice, and separated by a layer of dark chocolate ganache. Garnished with a dusting of cocoa powder and a single, hand-painted chocolate coffee bean.","menu_name":"Spiced Chocolate Symphony"},{"menu_description":"Five layers of our darkest chocolate cake, layered with espresso buttercream and topped with a white chocolate ganache.","menu_name":"Midnight In Paris Cake"},{"menu_description":"Three layers of decadent chocolate cake, with alternating layers of rich chocolate mousse and a hint of espresso.  Topped with chocolate shavings and a delicate chocolate swan.","menu_name":"Chocolate Swan Cake"},{"menu_description":"Five layers of moist chocolate cake, each infused with a different exotic spice, layered with a rich dark chocolate ganache and topped with a delicate chocolate curl.","menu_name":"Chocolate Concerto"},{"menu_description":"A Parisian cafe classic with a touch of opulence! Smooth, strong espresso given a velvety smooth texture with a dollop of rich, dark chocolate ganache. This drink is small but mighty, perfect for a quick pick-me-up or to accompany one of our exquisite chocolates.","menu_name":"Parisian Cafe Mocha"},{"menu_description":"A delightful miniature coffee cup, crafted from our richest dark chocolate, filled to the brim with smooth, freshly brewed espresso.","menu_name":"Chocolate Espresso Cup"},{"menu_description":"A playful twist on a classic combination! Our salted caramel dark chocolate bark is studded with pieces of buttery pretzels and a sprinkle of fleur de sel for a sweet and savory treat.","menu_name":"Salted Caramel Pretzel Bark"},{"menu_description":"A playful twist on a classic pairing!  Imagine biting into a light and airy pastry dusted with powdered sugar, only to discover a hidden center of decadent dark chocolate ganache, all perfectly balanced by the surprise of salty pretzel pieces. A delightful dance of sweet, salty, and a touch of bitterness. Perfect for sharing or indulging on your own.","menu_name":"Chocolate Covered Pretzel Choux au Craquelin"},{"menu_description":"A playful twist on a classic! Three fluffy churros served with a decadent dark chocolate dipping sauce, sprinkled with a hint of sea salt.","menu_name":"Churro Trio with Dark Chocolate Dip"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - with a hint of orange zest, served in an edible chocolate cup.","menu_name":"Dark & Zesty"},{"menu_description":"Crunchy, buttery, homemade graham cracker crust layered with creamy dark chocolate ganache and topped with a smoked sea salt meringue.","menu_name":"Dark Chocolate Sea Salt Meringue Pie"},{"menu_description":"A playful twist on a classic, our pretzel cones are dipped in our finest dark chocolate and sprinkled with sea salt.  Served with a side of our house-made salted caramel sauce.","menu_name":"Chocolate Dipped Pretzel Cones with Salted Caramel"},{"menu_description":"A whimsical journey for your taste buds!  White chocolate mousse infused with lavender, layered between delicate vanilla cookies. Topped with edible pansies and a sprinkle of pixie dust.","menu_name":"Enchanted Forest Dream"},{"menu_description":"A whimsical dance of textures and flavors! This dessert features a velvety smooth dark chocolate mousse infused with lavender, nestled atop a delicate almond biscuit. It is then crowned with a playful swirl of honey-lavender whipped cream and a sprinkling of edible lavender buds.","menu_name":"Lavender Dreamscape"},{"menu_description":"Three decadent chocolate hemispheres, each flavored with a different exotic spice, served with a dollop of saffron-infused whipped cream.","menu_name":"Spiced Chocolate Trio"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - on a bed of hazelnut dacquoise, topped with chocolate shavings and a sprinkle of sea salt.","menu_name":"Triple Chocolate Symphony"},{"menu_description":"Five layers of our smoothest dark chocolate ganache, separated by thin layers of our flakiest pastry.  Each layer is infused with a different spice, starting with cinnamon and progressing to cayenne.  A true adventure for your tastebuds!","menu_name":"Spiced Chocolate Napoleon"},{"menu_description":"A playful twist on a classic! Our signature dark chocolate sphere is dramatically melted tableside, revealing a hidden treasure of fluffy marshmallows and crunchy caramelized hazelnuts, all swimming in a pool of warm, salted caramel sauce.","menu_name":"Melting Chocolate Surprise"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white -  infused with exotic spices and served in a crispy, edible chocolate cup.  Topped with a delicate chocolate cigar.","menu_name":"Chocolate Trio"},{"menu_description":"A whimsical twist on a classic, featuring a delicate dark chocolate sphere filled with layers of rich chocolate mousse and a hint of sea salt.  Served with a side of espresso creme anglaise.","menu_name":"Deconstructed Chocolate Mousse"},{"menu_description":"Indulge in the rich aroma and taste of freshly brewed Colombian coffee, perfectly complemented by a handcrafted dark chocolate square infused with a hint of sea salt.","menu_name":" Parisian Midnight"},{"menu_description":"A whimsical journey for your taste buds!  Three decadent chocolate domes, each a different flavor, nestled on a bed of hazelnut praline crumble. Flavors include: Dark chocolate raspberry, white chocolate pistachio and milk chocolate caramel.","menu_name":"Chocolate Dream Trio"},{"menu_description":"A tower of our award winning chocolate fudge, stacked high and drizzled with raspberry coulis, topped with whipped cream.","menu_name":"Chocolate Fudge Tower"},{"menu_description":"A whimsical delight featuring our signature dark chocolate sphere, filled with a smooth salted caramel ganache and a hint of Himalayan pink salt.  Resting on a bed of delicate chocolate twigs, it's a feast for the eyes and the taste buds. ","menu_name":"Chocolate Caramel Orb"},{"menu_description":"A whimsical journey through a Parisian garden, featuring delicate chocolate butterflies perched atop a bed of edible flowers and verdant matcha moss.","menu_name":"Butterfly Garden"},{"menu_description":"Our handmade Parisian chocolate bark is a work of art.  A thin layer of dark chocolate is carefully crafted to look like an antique map of Paris, with landmarks and streets etched in white chocolate.  This delicate masterpiece is then sprinkled with edible gold flakes.","menu_name":"Parisian Chocolate Map"},{"menu_description":"A whimsical journey for your taste buds!  These succulent white chocolate flames are painted with edible red cocoa butter to give the appearance of fire.  The flames sit on a bed of hand crafted dark chocolate coals.  Please be aware that the flames will melt quickly, so savor them quickly!","menu_name":"Edible Chocolate Flames"},{"menu_description":"A delicate balance of bitter and sweet, our handcrafted dark chocolate bark is infused with crunchy roasted almonds and a touch of sea salt.  This bark is perfect for sharing or savoring on your own.","menu_name":"Dark Chocolate Almond Bark"},{"menu_description":"Five layers of our decadent chocolate cake are interwoven with a silky chocolate mousse, topped with a delicate chocolate ganache, and finished with a sprinkle of cocoa powder.  Best when shared.","menu_name":"Decadent Chocolate Dream"},{"menu_description":"A playful twist on a classic combination, this dessert features a mountain of fluffy, Madagascar vanilla bean marshmallows, toasted to perfection and nestled atop a bed of decadent dark chocolate mousse.  The mousse is then sprinkled with delicate flakes of smoked sea salt for a surprising salty kick.","menu_name":"Toasted Marshmallow Chocolate Mousse Peak"},{"menu_description":"A playful twist on a classic, this dessert features three miniature brioche donuts skewered and served with a rich, warm chocolate dipping sauce. ","menu_name":"Donut Brioche Skewers"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white chocolate, each infused with a hint of coffee and cardamom. Served with a side of chocolate-dipped coffee beans.","menu_name":"Triple Chocolate Coffee Dream"},{"menu_description":"A triple chocolate layered creation with a white, milk and dark chocolate ganache, finished with delicate chocolate curls.","menu_name":"Triple Chocolate Ganache"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - separated by thin, crispy wafers and topped with a drizzle of salted caramel and a sprinkle of cocoa nibs.","menu_name":"Sweet Surrender"},{"menu_description":"A warm chocolate lava cake, with a melted white chocolate center, topped with our house-made vanilla bean ice cream, and surrounded by a pool of raspberry coulis.","menu_name":"Molten White Chocolate Lava Cake"},{"menu_description":"A whimsical twist on a classic! Five fluffy, mini chocolate chip brioche donuts are served skewered and standing upright in a bed of creamy vanilla bean ice cream.  Topped with a drizzle of homemade salted caramel sauce and a sprinkle of cocoa powder, this dessert is as playful as it is delicious.  Perfect for sharing or indulging solo!","menu_name":"Doughnut Delight"},{"menu_description":"A shot of espresso with a dollop of whipped cream and a sprinkle of cocoa powder.","menu_name":"Classic Espresso"},{"menu_description":"Espresso with a touch of our homemade dark chocolate syrup, topped with whipped cream.  Served with a small piece of our hand made chocolate.","menu_name":"Dark Chocolate Dream Espresso"},{"menu_description":"A shot of espresso poured over a scoop of vanilla bean ice cream and topped with a sprinkle of cocoa powder.","menu_name":"Classic Affogato"},{"menu_description":"Indulge in a Parisian escape with our rich, dark chocolate espresso.  Served in a demitasse cup, this bold espresso is infused with notes of dark chocolate. Topped with a delicate chocolate curl for an elegant touch.","menu_name":"Parisian Chocolate Espresso"},{"menu_description":"Espresso with a double shot of our house blend, topped with a rich layer of melted dark chocolate shavings.","menu_name":"Molten Chocolate Espresso"},{"menu_description":"A rich and creamy blend of espresso and dark chocolate, topped with a delicate chocolate curl.","menu_name":"Parisian Mocha"},{"menu_description":"Indulge in a luxurious blend of rich, dark chocolate espresso, topped with a delicate layer of velvety cocoa foam. This sophisticated drink is perfect for those who appreciate the finer things in life.","menu_name":"Midnight Velvet Espresso"},{"menu_description":"A shot of espresso poured over a scoop of coconut ice cream, drizzled with our homemade dark chocolate sauce and sprinkled with toasted coconut flakes.","menu_name":"Coco Chanel"},{"menu_description":"A Parisian cafe classic with a chocolaty twist!  Espresso mixed with rich, dark chocolate, topped with a cloud of velvety smooth milk foam and a sprinkle of cocoa powder.","menu_name":"Parisian Delight"},{"menu_description":"A Parisian cafe classic with a twist; a double shot of espresso topped with a cloud of lightly sweetened whipped cream and a drizzle of rich dark chocolate shavings.","menu_name":"Parisian Cloud Espresso"},{"menu_description":"Indulge in a smooth, Parisian-style coffee experience with our signature blend. Sip and savor the rich aroma and robust flavor, perfectly complementing our exquisite chocolates.","menu_name":"Parisian Coffee"},{"menu_description":"Indulge in the rich aroma and taste of freshly brewed Colombian coffee, perfectly complemented by a delicate twist of orange zest.   Served hot and prepared to order.","menu_name":"Orange Zest Coffee"},{"menu_description":"Our signature dark roast coffee infused with the rich aroma of our finest dark chocolate. Topped with a sprinkle of cocoa powder.","menu_name":"Chocolate Infused Coffee"},{"menu_description":"Fresh brewed Ethiopian Yirgacheffe coffee with steamed milk.  Served with a small chocolate spoon.","menu_name":"Parisian Latte"},{"menu_description":"A double shot of espresso poured over a single scoop of our house-made vanilla bean ice cream and topped with a shot of our homemade salted caramel sauce.","menu_name":"Salted Caramel Affogato"},{"menu_description":"Freshly brewed dark roast coffee, infused with a hint of cardamom and topped with a delicate swirl of whipped cream. Served with a hand-crafted dark chocolate square, dusted with cocoa powder.","menu_name":"Cardamom Coffee Indulgence"},{"menu_description":"Rich, dark chocolate espresso with a hint of lavender and a sprinkle of cocoa nibs. Topped with a delicate lavender-infused whipped cream.","menu_name":"Lavender Espresso Dream"},{"menu_description":"Rich, dark chocolate espresso with a hint of lavender and topped with a delicate chocolate curl.","menu_name":"Midnight in Paris Mocha"},{"menu_description":"Rich, dark chocolate espresso with a hint of lavender and a sprinkle of cocoa powder. Topped with a delicate lavender flower.","menu_name":"Lavender Espresso"},{"menu_description":"Rich, dark chocolate espresso with a hint of lavender, topped with a delicate lavender-infused whipped cream.","menu_name":"Lavender Dream"},{"menu_description":"Espresso with a scoop of our house-made vanilla bean ice cream, topped with a drizzle of dark chocolate sauce and a sprinkle of cocoa powder.","menu_name":"Chocolate Affogato"},{"menu_description":"A layered symphony of espresso, dark chocolate ganache, and hazelnut cream, topped with a delicate chocolate curl.","menu_name":"Parisian Chocolate Layered Latte"},{"menu_description":"Our signature Parisian hot chocolate infused with a hint of lavender and topped with a delicate lavender-infused whipped cream. Served with a side of our handmade dark chocolate bark.","menu_name":"Lavender Dreams Hot Chocolate"},{"menu_description":"Three handcrafted, decadent chocolate bars. Flavors:  Black Lava Sea Salt, Madagascar Vanilla Bean, Espresso with Cacao Nibs. ","menu_name":"Trio of Chocolate Bars"},{"menu_description":"Indulge in the rich aroma and invigorating taste of our freshly brewed Parisian Roast.  Served in a classic mug.","menu_name":"Parisian Roast"},{"menu_description":"Our signature blend of Ethiopian Yirgacheffe and Costa Rican beans, brewed to perfection using a pour-over method. Served black or with a choice of milk and sugar.","menu_name":"Pour-Over Coffee"},{"menu_description":"Our signature espresso blended with a hint of lavender and topped with a cloud of frothed milk, reminiscent of a Parisian evening.","menu_name":"Parisian Twilight Latte"},{"menu_description":"Espresso with a shot of lavender syrup, topped with steamed milk and a sprinkle of dried lavender buds.","menu_name":"Lavender Latte"},{"menu_description":"Rich, dark chocolate espresso with a hint of cardamom, topped with a delicate chocolate curl.","menu_name":"Cardamom Chocolate Espresso"},{"menu_description":"Indulge in the perfect pairing for our exquisite chocolates. This Parisian cafe au lait offers a smooth blend of bold espresso and steamed milk, creating a velvety texture that complements the richness of our handcrafted treats.","menu_name":"Parisian Cafe Au Lait"},{"menu_description":"Indulge in a velvety smooth latte infused with the rich, smoky notes of our exclusive single-origin Rwandan coffee beans. Topped with a delicate cloud of microfoam, this latte is a symphony of flavor.","menu_name":"Volcano Coffee"},{"menu_description":"Escape to a Parisian cafe with our indulgent Lavender White Mocha. This creamy, dreamy beverage blends rich white chocolate with a hint of lavender, topped with whipped cream and a sprinkle of culinary lavender buds. ","menu_name":"Lavender White Mocha"},{"menu_description":"Our signature cold brew, steeped for 12 hours and infused with nitrogen for a velvety smooth texture, gets a touch of sweetness with our homemade salted caramel syrup. This is then topped with a delicate layer of light and airy cold foam, sprinkled with a touch of sea salt.","menu_name":"Salted Caramel Nitro Cold Brew"},{"menu_description":"A Parisian cafe classic with a velvety chocolate twist.  This rich and creamy hot chocolate is infused with our finest dark chocolate, topped with a mountain of whipped cream.","menu_name":"Parisian Hot Chocolate"},{"menu_description":"Indulge in a luxurious Parisian escape with our Signature Chocolate Cloud Cappuccino. This delightful creation blends velvety smooth espresso with steamed milk, topped with a generous dollop of whipped cream infused with our signature dark chocolate. But here's the twist: a delicate, hand-crafted chocolate cloud sits atop the cream, designed to melt slowly, releasing a symphony of flavors with every sip.","menu_name":"Signature Chocolate Cloud Cappuccino"},{"menu_description":"A symphony of textures and temperatures, this beverage features a shot of rich espresso poured over a scoop of vanilla bean ice cream, crowned with a cloud of lightly torched meringue.","menu_name":"Eiffel Cloud"},{"menu_description":"Indulge in the rich aroma and velvety texture of our Parisian Hot Chocolate, crafted with premium dark chocolate and a hint of cinnamon. ","menu_name":"Parisian Hot Cocoa"},{"menu_description":"Our signature iced coffee, blended with vanilla bean ice cream, and topped with a mountain of whipped cream, chocolate shavings, and a drizzle of salted caramel.","menu_name":"Parisian Iced Coffee Dream"},{"menu_description":"Indulge in a luxurious Parisian escape with our signature 'Chocolate Decadence' coffee.  This delightful creation combines rich, freshly brewed espresso with a velvety smooth dark chocolate ganache, topped with a delicate swirl of whipped cream and a sprinkle of cocoa powder.  A symphony of flavors that will tantalize your taste buds and leave you craving for more.","menu_name":"Chocolate Decadence Coffee"},{"menu_description":"A rich and creamy coffee enhanced with the subtle sweetness of our signature white chocolate. Topped with a delicate cloud of whipped cream.","menu_name":"White Chocolate Dream Latte"},{"menu_description":"A Parisian take on the classic mocha.  Espresso, dark chocolate, steamed milk  and topped with whipped cream and chocolate shavings. Small Batch Coffee Roasters.","menu_name":"Parisian Mocha "},{"menu_description":"Indulge in a decadent cup of velvety smooth hot chocolate, infused with our signature blend of single-origin cocoa beans.  Topped with a torched marshmallow and a sprinkle of sea salt.","menu_name":"Salted Caramel Hot Chocolate"},{"menu_description":"Freshly brewed Colombian coffee with a shot of rich, dark chocolate ganache. Topped with whipped cream and a sprinkle of cocoa powder.","menu_name":"Parisian Chocolate Ganache Coffee"},{"menu_description":"A rich and creamy coffee blended with our homemade dark chocolate sauce, topped with a mountain of whipped cream and chocolate shavings.","menu_name":"Parisian Chocolate Dream"},{"menu_description":"A Parisian escape in a cup. Rich, dark chocolate espresso is blended with a hint of lavender and topped with a delicate lavender-infused whipped cream. Pure indulgence.","menu_name":"Lavender Midnight Mocha"},{"menu_description":"A warm, intensely chocolatey drink made with single-origin dark chocolate, a hint of orange zest, and a shot of espresso. Topped with a cloud of vanilla bean infused whipped cream and a sprinkle of cocoa powder.","menu_name":"Parisian Noir Orange Dream"},{"menu_description":"A Parisian Escape in Every Sip: Indulge in our rich, dark chocolate espresso, topped with a delicate hand-piped chocolate Eiffel Tower. Perfect for those seeking a luxurious coffee experience.","menu_name":"Parisian Chocolate Tower Espresso"},{"menu_description":"Indulge in a Parisian escape with our signature drink, featuring rich, dark chocolate notes interwoven with the subtle sweetness of caramel, topped with a delicate swirl of whipped cream and a sprinkle of cocoa. Pure bliss in a cup.","menu_name":"Parisian Chocolate Caramel Dream"},{"menu_description":"Indulge in a symphony of flavors with our signature coffee creation.  We combine rich, freshly brewed espresso with a touch of vanilla syrup, then top it with a cloud of velvety smooth vanilla bean cold foam.  Finally, we drizzle it with a silky smooth chocolate ganache for the ultimate coffee experience.","menu_name":"Parisian Vanilla Bean Dream"},{"menu_description":"Our signature Parisian hot chocolate taken to the next level. Steamed milk infused with our secret blend of spices, poured over single-origin dark chocolate, and topped with a mountain of house-made vanilla bean marshmallows.","menu_name":"Parisian Hot Chocolate Extravaganza"},{"menu_description":"Rich, dark chocolate shavings gently infused into a smooth, velvety latte. Topped with a delicate, hand-crafted chocolate garnish. ","menu_name":"Midnight Chocolate Latte"},{"menu_description":"Our signature cold brew, steeped for 20 hours, infused with a shot of rich dark chocolate sauce and a hint of lavender. Topped with a delicate cloud of frothed milk.","menu_name":"Midnight in Paris"},{"menu_description":"Indulge in the rich aroma and taste of our Lavender Mocha, a Parisian dream in a cup. Steamed milk infused with French lavender syrup, our signature espresso blend, and a touch of vanilla, topped with a delicate lavender sprig. ","menu_name":"Lavender Mocha"},{"menu_description":"Indulge in the rich aroma and bittersweet symphony of dark chocolate espresso, topped with a cloud of velvety smooth vanilla bean infused cream. This luxurious drink is perfect for those who appreciate the finer things in life.","menu_name":"Choco-Espresso"},{"menu_description":"A delightful twist on the classic latte, our Lavender Vanilla Bean Latte is as pleasing to the eye as it is to the palate. Steamed milk is infused with the subtle floral essence of lavender and the rich sweetness of vanilla bean, then combined with our signature espresso. Topped with a delicate lavender sprig, it's a sensory journey in every sip.","menu_name":"Lavender Vanilla Bean Latte"},{"menu_description":"A single, exquisitely crafted dark chocolate truffle, infused with delicate lavender blossoms and a hint of sea salt. Designed to resemble a blooming flower.","menu_name":"Lavender Bloom Truffle"},{"menu_description":"A sphere of dark chocolate filled with our house blend of dark roasted coffee infused ganache. Topped with a white chocolate swirl.","menu_name":"Coffee Bean Dream"},{"menu_description":"A single, intensely rich dark chocolate truffle, infused with notes of bergamot and topped with a delicate sprinkle of edible gold dust.","menu_name":"Midnight in Paris Truffle"},{"menu_description":"A delicate milk chocolate sphere, filled with a smooth coffee ganache and a hint of hazelnut. Decorated with edible gold dust and a single coffee bean.","menu_name":" Parisian Twilight"},{"menu_description":"A smooth and velvety chocolate dome, infused with the subtle spice of saffron, encasing a hidden heart of liquid salted caramel.","menu_name":"Saffron-Infused Chocolate Sphere"},{"menu_description":"A playful twist on a classic.  Vanilla bean ice cream dipped in our finest dark chocolate, rolled in crushed hazelnuts, and topped with a pinch of sea salt.  Made with our handcrafted chocolate.","menu_name":"Chocolate Covered Ice Cream Bon Bon"},{"menu_description":"Three handcrafted dark chocolate spheres, each filled with a different exotic mousse:  Black Sesame, Cardamom Rosewater, and Pistachio Saffron - an explosion of flavor with an artful touch. ","menu_name":"Parisian Nights Chocolate Trio"},{"menu_description":"Three handcrafted chocolates infused with matcha, black sesame, and ginger, offering an symphony of Eastern flavors.","menu_name":"Eastern Indulgence"},{"menu_description":"Three chocolate ganache squares flavored with lavender, Earl Grey tea, and black truffle oil.","menu_name":"Parisian Ganache Trio"},{"menu_description":"Three chocolate truffles infused with the essence of Earl Grey tea, coated in a delicate layer of white chocolate and sprinkled with edible gold dust.","menu_name":"Earl Grey & Gold Truffles"},{"menu_description":"A whimsical trio of decadent chocolate spheres, each hand-painted to resemble planets: swirling emerald matcha white chocolate for Earth, shimmering ruby raspberry for Mars, and deep, dark chocolate with edible silver for the Moon.  Served on a slate slab.","menu_name":"Cosmic Trio"},{"menu_description":"Three handcrafted chocolates flavored with Saffron, Rose Water and Cardamom. Topped with delicate edible gold leaf.","menu_name":"Persian Dreams"},{"menu_description":"Three handcrafted dark chocolate spheres, each hand-painted to resemble a celestial body, encasing a smooth, rich ganache infused with a hint of Earl Grey tea.  Served with a side of fresh raspberries.","menu_name":"Celestial Trio"},{"menu_description":"Three handmade dark chocolate hemispheres each with a unique ganache filling: lavender infused honey, black truffle, and espresso cardamom. Garnished with edible gold leaf.","menu_name":"Parisian Hemisphere Trio"},{"menu_description":"Three handmade, painted chocolates filled with pistachio and cardamom ganache, and topped with edible gold.","menu_name":"Pistachio Cardamom Jewels"},{"menu_description":"Three decadent chocolate truffles infused with lavender and honey. Topped with crystallized lavender and a drizzle of honey.","menu_name":"Lavender Honey Truffle"},{"menu_description":"Three perfectly crafted chocolate spheres, each infused with a different exotic spice blend. Flavors include star anise and cinnamon, cardamom and ginger, and chili and black pepper.","menu_name":"Spiced Chocolate Sphere Trio"},{"menu_description":"Five uniquely flavored truffles:  Espresso Yourself (Coffee Ganache, Dark Chocolate), Parisian Night (Blackcurrent Ganache, Milk Chocolate), Golden Hour (Salted Caramel Ganache, White Chocolate), Berry Beautiful (Raspberry Hibiscus Ganache, Ruby Chocolate), and Mint to Be (Mint Ganache, Dark Chocolate).","menu_name":"Parisian Truffle Flight"},{"menu_description":"Five uniquely flavored chocolate truffles.  Each truffle is a delicate work of art. Flavors include:  blackberry lavender, pistachio rosewater, salted caramel espresso, raspberry ginger, and strawberry balsamic.","menu_name":"Chocolate Tasting Flight"},{"menu_description":"Five handcrafted dark chocolate squares, each infused with a different exotic spice, creating a symphony of flavors from sweet to smoky.","menu_name":"Parisian Spice Journey"},{"menu_description":"Five individually crafted dark chocolate pieces, each infused with a different exotic spice, paired with a small cup of rich, dark drinking chocolate. - Cardamom & Black Pepper - Star Anise & Cinnamon - Chipotle & Chili - Ginger & Nutmeg - Saffron & Turmeric","menu_name":"Spice Trader's Chocolate Journey"},{"menu_description":"Five individually crafted dark chocolate squares, each infused with a different exotic spice, like saffron, cardamom, and black lava salt. Designed to ignite your palate and transport you to the bustling souks of Marrakech.","menu_name":"Moroccan Spice Journey"},{"menu_description":"A six pack of our most popular hand-made spicy chocolates.","menu_name":"Spicy Chocolate Six Pack"},{"menu_description":"Indulge in the rich aroma and taste of our freshly brewed coffee, perfectly complementing the nuanced flavors of our hand-crafted chocolates.","menu_name":"Parisian Coffee Pairing"},{"menu_description":"A whimsical tower of dark and white chocolate profiteroles, filled with a light and creamy coffee infused Chantilly cream, artfully arranged on a bed of delicate chocolate shavings and crowned with a spun sugar masterpiece, reminiscent of the Eiffel Tower. Perfect for sharing or a decadent treat for one.","menu_name":"Eiffel Tower Profiterole Tower"},{"menu_description":"A tower of crispy, buttery puff pastry layered with rich dark chocolate ganache and a hint of sea salt. Designed for sharing, it's an architectural and decadent masterpiece. ","menu_name":"Chocolate Paris Tower"},{"menu_description":"A tower of dark, milk, and white chocolate mousse, layered with crunchy hazelnut praline and delicate chocolate flakes. This dessert is an architectural masterpiece, meant to be shared and savored.","menu_name":"Eiffel Tower Mousse"},{"menu_description":"A whimsical journey for two. This dessert features a cloud of light-as-air brioche donuts, skewered and drizzled with dark, milk, and white chocolate, and served with a side of rich chocolate dipping sauce.","menu_name":"Sweet Ascent"},{"menu_description":"A whimsical journey for two! Five miniature hot air balloon shaped chocolate cups filled with pistachio, cardamom, and rose infused ganache. Served alongside a miniature pot of our richest hot chocolate for dipping.","menu_name":"Chocolate Balloon Ride for Two"},{"menu_description":"A playful twist on a classic, featuring a tower of light and airy churros dusted with cinnamon sugar, served with a side of rich dark chocolate sauce for dipping.","menu_name":"Churro Tower"},{"menu_description":"A whimsical twist on a classic. This dessert features a tower of light and airy churros, dusted with cinnamon sugar and served with a rich and decadent dark chocolate dipping sauce. Perfect for sharing!","menu_name":"Churro Tower of Dreams"},{"menu_description":"A whimsical twist on a classic!  This dessert features a tower of three fluffy chocolate chip cookies, layered with creamy vanilla bean ice cream, and drizzled with rich dark chocolate sauce.  It's perfect for sharing or indulging on your own.","menu_name":"Chocolate Chip Cookie Ice Cream Tower"},{"menu_description":"A tower of crispy, buttery waffle bites, stacked high and drizzled with melted dark chocolate. Perfect for sharing or indulging solo.","menu_name":"Chocolate Waffle Tower"},{"menu_description":"A symphony of textures and flavors! Smooth coffee ganache meets crunchy pretzel pieces, all coated in luxurious dark chocolate.","menu_name":"Salted Caramel Coffee Dreams"},{"menu_description":"Three layered chocolate bar.  Each layer a different percentage of cocoa, topped with cocoa nibs.","menu_name":"Triple Layer Bar"},{"menu_description":"A whimsical twist on a classic dessert! Our choux pastry is filled with a light and creamy lavender-infused white chocolate ganache, topped with a delicate lavender flower.","menu_name":"Lavender White Chocolate Cream Puffs"},{"menu_description":"Three decadent chocolate truffles infused with the rich aroma of Parisian coffee, served with a dollop of fresh whipped cream and a sprinkle of cocoa powder.","menu_name":"Parisian Coffee Truffle Trio"},{"menu_description":"A playful twist on a classic dessert, this dessert features three miniature waffle cones dipped in our finest dark chocolate. Each cone is then filled with a silky smooth coffee-infused mousse and topped with a delicate chocolate curl. ","menu_name":"Chocolate Dipped Coffee Mousse Cones"},{"menu_description":"Five individually crafted dark chocolate squares, each infused with a different exotic spice, like saffron, cardamom, and star anise. Designed to be savored slowly, allowing the flavors to unfold on the palate.","menu_name":"Spice Route Tasting Set"},{"menu_description":"Five (5) rich dark chocolate ganache cups infused with cardamom and a hint of black pepper. Each cup is topped with an edible gold dusted star.","menu_name":"Cosmic Cups"},{"menu_description":"Five (5) miniature chocolate bars each hand-painted to resemble a different precious stone.","menu_name":"Edible Gems"},{"menu_description":"Five handmade bonbons with a contemporary twist.  Flavors include exotic spices and herbs paired with unique fruits.","menu_name":"Avant-Garde Bonbons"},{"menu_description":"A whimsical dance of dark chocolate elegance.  Five handcrafted, dark chocolate spheres, each hand-painted with edible gold, encasing a surprise filling of balsamic caramel and a hint of black lava salt. ","menu_name":"Golden Night"},{"menu_description":"Five (5) uniquely flavored truffles crafted with rare Venezuelan cocoa beans and dusted with edible gold. ","menu_name":"Parisian Gold Truffles"},{"menu_description":"Five (5) miniature chocolate bars, each infused with a different exotic spice, served with a small pot of rich dark chocolate dipping sauce.","menu_name":"Spice Trade Chocolate Flight"},{"menu_description":"Five (5) pieces of dark chocolate ganache infused with cardamom and a hint of black pepper, topped with crunchy cocoa nibs, resembling a miniature abstract sculpture.","menu_name":"Spiced Ganache Treasures"},{"menu_description":"Five pieces of dark chocolate, each infused with a different exotic pepper, paired with a velvety dark chocolate mousse and a sprinkle of sea salt.","menu_name":"Parisian Spice"},{"menu_description":"A playful twist on a classic! Five miniature chocolate hot air balloons, each handcrafted with a different single-origin dark chocolate. The baskets are filled with a light and airy chocolate mousse.","menu_name":"Chocolate Hot Air Balloons"},{"menu_description":"Five decadent pieces of hand-painted dark chocolate with unexpected flavor combinations. Each piece is a tiny work of art, meant to be savored slowly. Flavors change seasonally.","menu_name":"Colorful Confections"},{"menu_description":"Five individually crafted chocolates, each infused with a different exotic spice, like saffron, cardamom, and pink peppercorn. Designed to ignite your palate and transport you to a Parisian spice market.","menu_name":"Spice Trader's Collection"},{"menu_description":"Five (5) hand-painted chocolates filled with a silky smooth pistachio ganache and candied rose petals. Designed to look like precious gemstones, these chocolates are almost too beautiful to eat.","menu_name":"Emerald of the Seine"},{"menu_description":"Five (5) decadent, dark chocolate truffles infused with a hint of smoky Lapsang Souchong tea, dusted with edible gold, and presented on a sleek black slate.","menu_name":"Smoked Tea Truffle Quintet"},{"menu_description":"Five (5) decadent, Parisian-inspired chocolate bonbons, each a tiny work of art. Flavors include Lavender Honey, Salted Caramel with Black Sea Salt, Raspberry Rosewater, Espresso with Cacao Nibs, and Champagne infused white chocolate. ","menu_name":"Parisian Dream Bonbons"},{"menu_description":"Five (5) hand-painted bonbons, inspired by the architecture of Paris, with flavors like Lavender Honey Caramel, Black Truffle Ganache, and Salted Butter Caramel.","menu_name":"Parisian Promenade"},{"menu_description":"Five (5) handcrafted chocolates shaped and painted to resemble colorful macarons. Each macaron-chocolate is filled with a different ganache: lavender-honey, vanilla bean, raspberry rose water, pistachio, and salted caramel.","menu_name":"Chocolate Macaron Delights"},{"menu_description":"Five dark chocolate squares, each filled with a different exotic spice ganache, topped with an edible gold leaf. Inspired by the ancient spice route, this selection is a journey for the senses.","menu_name":"Spice Route Selection"},{"menu_description":"Five (5) hand-painted bonbons, each featuring a different Parisian landmark, crafted with a unique flavor combination.","menu_name":"Parisian Landmarks Collection"},{"menu_description":"Five (5) handcrafted chocolates featuring a different region of the world.  Each piece is designed to take you on a journey.  Vegan and nut free options available upon request.","menu_name":"Chocolate World Tour"},{"menu_description":"Five pieces of dark chocolate hand-painted to resemble famous works of art. Flavors include rosemary sea salt, lavender honey, raspberry rose, black sesame ginger, and chili lime.","menu_name":"Edible Masterpieces"},{"menu_description":"Five (5) decadent, hand-painted bonbons, featuring unique flavor combinations inspired by the art and flavors of Paris. Presented on a sleek, black serving tray.","menu_name":"Landmark Lockets"},{"menu_description":"Five pieces of our surprise handcrafted chocolates. Flavors change daily.","menu_name":"Chocolate Discovery"},{"menu_description":"Five pieces of dark chocolate, each infused with a different exotic spice. Flavors include cardamom rose, saffron ginger, black pepper cinnamon, star anise clove, and turmeric nutmeg.","menu_name":"Spice Route Chocolate Flight"},{"menu_description":"Five chocolate sticks flavored with cardamom, rose, saffron, ginger, and pistachio, representing the aromatic bazaars of the East.","menu_name":"Spice Bazaar Chocolate Sticks"},{"menu_description":"Five pieces of dark chocolate infused with lavender and honey, topped with edible gold flakes.","menu_name":"Lavender Honey Dusk"},{"menu_description":"Five (5) hand-rolled dark chocolate truffles infused with cardamom and a hint of black pepper.  Served with a side of our house blend dark chocolate dipping sauce.","menu_name":"Parisian Nights"},{"menu_description":"A playful twist on a classic! Five miniature waffle cones, each hand-dipped in our finest dark, milk, or white chocolate, then adorned with colorful sprinkles and a drizzle of complementary flavored chocolate.","menu_name":"Chocolate Dipped Waffle Cone Bites"},{"menu_description":"Five hand-crafted truffles:  Dark chocolate sea salt, Lavender infused white chocolate, Espresso with dark chocolate ganache, pistachio and cardamom, milk chocolate with smoky bourbon.","menu_name":"Parisian Truffle Collection"},{"menu_description":"A whimsical twist on a classic!  Parisian dark chocolate squares filled with homemade toasted marshmallow fluff and graham cracker crumbs, all coated in a delicate layer of edible gold dust. Perfect for sharing or a luxurious solo treat!","menu_name":"Golden Parisian S'mores"},{"menu_description":"A whimsical assortment of chocolate bonbons, each hand-painted to resemble a different Parisian landmark like the Eiffel Tower, the Louvre Pyramid, and the Arc de Triomphe. Flavors include raspberry ganache, salted caramel, and hazelnut praline.","menu_name":"Parisian Dreamscape Bonbons"},{"menu_description":"A cosmic dance of dark chocolate infused with ginger and cayenne pepper, sprinkled with edible gold dust.","menu_name":"Cosmic Cacao"},{"menu_description":"A whimsical journey for your taste buds! Smooth milk chocolate infused with warming chai spices, crafted into playful animal shapes. Each piece is a miniature work of art, perfect for sharing or indulging in a moment of pure delight.","menu_name":"Chai Chocolate Zoo"},{"menu_description":"Indulge in a symphony of flavors as rich, dark chocolate ganache meets the gentle sweetness of a cloud-like marshmallow center, all nestled within a buttery croissant.  Finished with a dusting of powdered sugar and a drizzle of chocolate.","menu_name":"Chocolate Marshmallow Croissant"},{"menu_description":"A flight of four, single origin dark chocolate squares, each paired with a curated cheese and condiment.","menu_name":"Chocolate & Cheese Flight"},{"menu_description":"Rich, dark chocolate ganache infused with the subtle spice of black pepper, encased in a delicate chocolate shell, served with a side of fresh whipped cream.","menu_name":"Black Pepper Ganache Sphere"},{"menu_description":"Indulge in the exquisite pairing of our darkest, richest hot chocolate, crafted with single-origin Venezuelan Criollo beans, and a single, hand-painted chocolate truffle infused with fragrant Earl Grey tea. ","menu_name":"Dark Chocolate Decadence"},{"menu_description":"A whimsical journey into the heart of chocolate artistry.  A savory blend of rich dark chocolate infused with smoky Lapsang Souchong tea leaves, topped with a delicate chocolate sculpture of the Eiffel Tower dusted with edible gold.","menu_name":"Parisian Twilight Delight"},{"menu_description":"A whimsical journey for two through a miniature chocolate garden. Delicate chocolate butterflies flutter around a bed of edible flowers, while a hidden well of rich chocolate mousse awaits discovery. ","menu_name":"Chocolate Garden of Dreams"},{"menu_description":"A whimsical journey for two.  Handmade dark chocolate hot air balloon, filled with our house-made vanilla bean ice cream, covered with a delicate spun sugar cloud.","menu_name":"Hot Air Balloon Adventure"},{"menu_description":"Five layers of our richest dark chocolate ganache, separated by thin layers of hazelnut dacquoise, coated in a shimmering dark chocolate glaze, and topped with a delicate chocolate hazelnut twig.","menu_name":"Chocolate Hazelnut Symphony"},{"menu_description":"A tower of three chocolate hemispheres, each painted with edible gold and filled with different flavored chocolate mousse:  dark chocolate espresso,  white chocolate raspberry,  and milk chocolate hazelnut. ","menu_name":"The Golden Trio"},{"menu_description":"A whimsical journey for two!  Three handmade chocolate spheres filled with our gourmet ice cream: Madagascar Vanilla, Pistachio Rosewater, and Salted Caramel.  Served with a trio of dipping sauces:  Hot Chocolate Ganache, Raspberry Coulis, and Whipped Espresso Cream.  Designed for sharing and savoring.","menu_name":"Chocolate Sphere Surprise"},{"menu_description":"A whimsical tower of dark and white chocolate spheres, delicately balanced and filled with a surprise assortment of salted caramel, hazelnut praline, and raspberry ganache.  Served with fresh raspberries and a drizzle of raspberry coulis.","menu_name":"Eiffel Tower of Chocolate Delights"},{"menu_description":"A whimsical journey for two!  Six miniature chocolate hot air balloons, each filled with a different flavour of mousse: raspberry, passionfruit, hazelnut, salted caramel, pistachio, and coffee.","menu_name":"Chocolate Hot Air Balloon Flight"},{"menu_description":"Five layers of our handmade chocolate, each infused with a different exotic spice, stacked high and drizzled with a chili-infused dark chocolate ganache.  This dessert is not overly spicy, but has a subtle kick to it! ","menu_name":"Aztec's Spicy Chocolate Tower"},{"menu_description":"Three decadent chocolate lava cakes, crafted with our finest dark chocolate, and a hint of sea salt.  Topped with powdered sugar and served with a side of vanilla bean ice cream.","menu_name":"Trio of Chocolate Lava Cakes"},{"menu_description":"A whimsical journey for two! Dive into a heart-shaped dark chocolate box filled with handcrafted chocolates. Each piece is a unique flavor explosion, featuring intriguing ingredients like black lava salt, pink peppercorn, and edible gold flakes. Indulge in the unexpected and savor the artistry of each bite.","menu_name":"Ethereal Hearts Desire"},{"menu_description":"A whimsical assortment of hand-crafted chocolates, each shaped and decorated like a Parisian garden element. Indulge in crunchy hazelnut praline cobblestones, delicate dark chocolate lavender flowers, and smooth milk chocolate park benches.","menu_name":"Edible Parisian Garden"},{"menu_description":"A whimsical assortment of handcrafted chocolates designed to resemble a painter's palette. Each chocolate is a unique flavor and color, crafted with the finest ingredients.","menu_name":"The Artist's Palette"},{"menu_description":"A miniature masterpiece featuring a dark chocolate dome filled with smooth coffee ganache, sitting atop a crunchy hazelnut praline base.","menu_name":"Coffee Ganache Dome"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - intricately swirled together and topped with delicate chocolate curls.","menu_name":"Cocoa Cream Delight"},{"menu_description":"A playful twist on a classic pairing - warm, gooey chocolate chip cookies meet a smooth and decadent dark chocolate mousse. Imagine biting into a cloud of chocolate heaven. This is best enjoyed with a warm cup of coffee! ","menu_name":"Chocolate Chip Cookie Mousse Duo"},{"menu_description":"Three layers of decadent dark chocolate mousse, separated by layers of hazelnut dacquoise and topped with a silky chocolate ganache.","menu_name":"Triple Chocolate Hazelnut Delice"},{"menu_description":"A sphere of dark chocolate filled with layers of red velvet cake, cream cheese frosting, and a hint of raspberry.","menu_name":"Red Velvet Surprise"},{"menu_description":"A whimsical chocolate sphere, filled with edible gold flakes and a surprise center of popping candy. Resting on a bed of spun sugar that resembles a bird's nest.","menu_name":"Golden Egg Surprise"},{"menu_description":"A single, intensely rich dark chocolate dome filled with a smooth coffee ganache and topped with a delicate white chocolate swirl.","menu_name":"Ebony & Ivory"},{"menu_description":"A whimsical delight! Dive into a playful dome of rich milk chocolate, hiding a secret center of marshmallow fluff and a sprinkle of sea salt. As you crack through, discover a hidden treasure of milk chocolate coffee beans, delivering a playful crunch and caffeine kick.","menu_name":"Parisian Surprise"},{"menu_description":"A rich and silky dark chocolate mousse, infused with aromatic lavender and topped with a delicate honey-comb crunch. This dessert is a sensory delight that will transport you to a Parisian cafe.","menu_name":"Midnight in Paris Mousse"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - artfully swirled together, creating a captivating marble effect. Topped with delicate chocolate curls and a sprinkle of cocoa powder. This dessert is a symphony of flavor and elegance.","menu_name":"Parisian Chocolate Symphony"},{"menu_description":"A decadent and rich chocolate mousse, infused with the finest Earl Grey tea. Topped with a delicate layer of white chocolate infused with bergamot.","menu_name":"Earl Grey & Bergamot Chocolate Symphony"},{"menu_description":"A  swirl of decadent dark chocolate ganache infused with fragrant Earl Grey tea, encased in a buttery shortbread crust. Topped with delicate, hand-painted chocolate curls and a sprinkle of edible gold dust.","menu_name":"Earl Grey Chocolate Tartlet"},{"menu_description":"A symphony of textures and flavors, featuring a velvety chocolate mousse infused with rich black pepper, encased in a delicate chocolate sphere. Garnished with a sprinkle of gold dust and a single cocoa nib.","menu_name":"Black Pepper & Gold Sphere"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white, separated by layers of hazelnut dacquoise and topped with chocolate shavings.","menu_name":"Triple Chocolate Hazelnut Dream"},{"menu_description":"A whimsical dance of dark chocolate mousse infused with lavender, nestled atop a buttery almond biscuit. Experience the surprise of black currant jelly pockets and a sprinkle of delicate lavender buds.","menu_name":"Midnight in Provence"},{"menu_description":"A whimsical dance of textures and tastes! Smooth dark chocolate mousse, nestled on a bed of crunchy hazelnut praline, all embraced by a delicate spun sugar sphere. The sphere cracks open, revealing the mousse and praline, creating a delightful symphony of flavors and visual delight.","menu_name":"Ethereal Chocolate Sphere"},{"menu_description":"A delicate chocolate sphere filled with a warm chocolate ganache.  Served with a side of fresh raspberries.","menu_name":"Melting Chocolate Sphere"},{"menu_description":"A whimsical twist on a classic. Smooth, creamy chocolate mousse infused with the subtle floral notes of lavender, nestled inside a dark chocolate shell, resembling a blooming flower. Garnished with edible gold dust.","menu_name":"Lavender Bloom"},{"menu_description":"A decadent chocolate sphere, painted with edible gold, encasing a hidden surprise.  As warm salted caramel sauce is poured over the sphere, it slowly melts away to reveal a scoop of vanilla bean ice cream, topped with a delicate chocolate butterfly.","menu_name":"Golden Orb Surprise"},{"menu_description":"Three layers of our darkest chocolate mousse, separated by thin layers of our homemade salted caramel and topped with a delicate chocolate cage filled with cocoa nibs.","menu_name":"Triple Chocolate Decadence"},{"menu_description":"A warm, fudgy brownie topped with a scoop of vanilla bean ice cream, surrounded by edible chocolate shards, and drizzled with raspberry coulis.","menu_name":"Chocolate Decadence"},{"menu_description":"A whimsical and decadent chocolate sphere, delicately hand-painted to resemble a swirling galaxy. As you crack it open, discover a hidden treasure of dark chocolate mousse infused with smoky Laphroaig whisky and a sprinkle of sea salt.","menu_name":"Cosmic Chocolate Sphere"},{"menu_description":"Five layers of decadent dark, milk and white chocolate mousse, intricately crafted to resemble a blooming Parisian rose. Garnished with edible gold flakes and served with a side of raspberry coulis.","menu_name":"Chocolate Rose"},{"menu_description":"Three chocolate domes, each painted with vibrant cocoa butter, encasing layers of mousse: dark chocolate with a hint of chili, milk chocolate infused with lavender, and white chocolate with a touch of pistachio.","menu_name":"Trio of Textures"},{"menu_description":"A decadent chocolate sphere filled with layers of hazelnut dacquoise, salted caramel sauce, and a hidden center of dark chocolate ganache. Served with a side of vanilla bean ice cream and drizzled with a warm chocolate sauce.","menu_name":"Melting Chocolate Sphere Surprise"},{"menu_description":"A  tower of delicate chocolate and hazelnut macarons, filled with rich dark chocolate ganache and sprinkled with edible gold dust. It's served over a bed of raspberry coulis, for a touch of tartness.","menu_name":"Golden Macaron Tower"},{"menu_description":"A tower of three chocolate mousse cups - dark chocolate, milk chocolate and white chocolate, all sitting on a crispy almond base. Decorated with chocolate curls and a drizzle of salted caramel.","menu_name":"Triple Chocolate Mousse Tower"},{"menu_description":"A tower of three decadent chocolate mousse cups, each infused with a different exotic spice: star anise, cardamom, and pink peppercorn. Topped with a delicate chocolate curl and a sprinkle of edible gold dust for a touch of Parisian elegance.","menu_name":"Spice Tower Mousse Trio"},{"menu_description":"Three layers of decadent dark chocolate mousse, each infused with a different exotic spice, sit atop a crunchy hazelnut praline base. This dessert is then crowned with a hand-painted chocolate dome resembling a golden Faberge egg, hiding a secret center of tart raspberry coulis.","menu_name":"Spiced Chocolate Faberge Egg"},{"menu_description":"A whimsical twist on a classic! Our chocolate sphere, handcrafted to resemble a melting art palette, is filled with layers of decadent chocolate mousse, crunchy chocolate pearls, and a hint of sea salt. It's then dramatically melted tableside with hot salted caramel sauce for a truly unforgettable dessert experience.","menu_name":"Melting Palette"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - separated by thin, crispy wafers and topped with a drizzle of salted caramel and a sprinkle of cocoa nibs. This dessert is served alongside a miniature sculpture made of our finest dark chocolate.","menu_name":"Mousse au Chocolat Couvert"},{"menu_description":"Three layers of our darkest chocolate mousse, separated by thin layers of hazelnut dacquoise and topped with a delicate chocolate cage.","menu_name":"Rich Indulgence"},{"menu_description":"Decadent layers of dark chocolate mousse and hazelnut dacquoise, adorned with delicate chocolate curls and a drizzle of Frangelico liqueur.","menu_name":"Parisian Midnight"},{"menu_description":"Three layers of decadent chocolate mousse, each infused with a different exotic spice, nestled between layers of delicate chocolate cake and topped with a hand-painted chocolate swirl.","menu_name":"Triple Chocolate Spice Deception"},{"menu_description":"Three layers of our finest dark, milk, and white chocolate mousse, delicately crafted to resemble elegant Parisian hats. Garnished with edible gold leaf and served with a side of fresh raspberries.","menu_name":"Chocolate Parisian Hats"},{"menu_description":"Three layers of our decadent chocolate mousse - dark, milk and white chocolate, interwoven with crispy, feuilletine flakes. Garnished with delicate chocolate curls and a drizzle of raspberry coulis.","menu_name":"Triple Chocolate Mousse Parfait"},{"menu_description":"Three layers of dark chocolate mousse, white chocolate mousse, and milk chocolate mousse, topped with chocolate shavings and served with a side of raspberry coulis.","menu_name":"Triple Chocolate Mousse Delight"},{"menu_description":"A playful twist on a classic! Creamy dark chocolate mousse infused with the subtle spice of black pepper, nestled inside a delicate, gluten-free almond flour tart shell. Garnished with a sprinkle of sea salt and a drizzle of high-end olive oil.","menu_name":"Black Pepper Chocolate Dream"},{"menu_description":"Three layers of our luxurious dark, milk, and white chocolate mousse, flavored with espresso and cardamom. Topped with a delicate chocolate cigarette and a dusting of cocoa powder.","menu_name":"Coffee Crunch"},{"menu_description":"Five layers of our richest dark chocolate mousse, separated by layers of hazelnut dacquoise and a hint of espresso.  Finished with chocolate curls and a drizzle of dark chocolate ganache.","menu_name":"Decadent Chocolate Hazelnut Symphony"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white, each infused with a hint of orange blossom.","menu_name":"Parisian Twilight Mousse"},{"menu_description":"A playful twist on a classic, this dessert features a warm, fudgy brownie shaped like a steaming cup of coffee, complete with a delicate white chocolate 'foam' and a side of coffee bean-shaped chocolate truffles.","menu_name":"Coffee Break Brownie"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white, separated by thin, crispy wafers. Topped with a delicate chocolate curl and a sprinkle of cocoa powder.","menu_name":"Layered Chocolate Mousse"},{"menu_description":"A whimsicle blend of white chocolate and pistachio, crafted to resemble the elegant curves of the Eiffel Tower. Served with a side of fresh raspberries.","menu_name":"Eiffel Tower Delight"},{"menu_description":"Three layers of decadent chocolate mousse - dark, milk, and white - artfully swirled together, and topped with a single, hand-painted chocolate curl. Served with a side of freshly whipped cream.","menu_name":"Parisian Pâtisserie Perfection"},{"menu_description":"Five layers of our richest dark chocolate mousse, separated by thin, crispy wafers and topped with a delicate chocolate filigree.","menu_name":"Chocolate Decadence Tower"},{"menu_description":"A delicate balance of bitter and sweet, this Parisian tower is crafted with layers of dark and white chocolate, designed to be shared or savored solo.","menu_name":"Eiffel Tower"},{"menu_description":"Four layers of our decadent chocolate cake, each infused with a different exotic spice, enrobed in a dark chocolate ganache and topped with a delicate chocolate filigree.","menu_name":"Parisian Spice Rhapsody"},{"menu_description":"A decadent and rich chocolate cake, layered with a coffee infused ganache, and topped with a delicate chocolate glaze. Perfect for sharing","menu_name":"Espresso Chocolate Dream Cake"},{"menu_description":"A whimsical journey for two! Five miniature chocolate hot air balloons, each filled with a different flavored mousse (e.g., raspberry, hazelnut, pistachio), served alongside a cloud of cotton candy.","menu_name":"Chocolate Balloon Ride"},{"menu_description":"Five handcrafted chocolate bonbons, each a tiny work of art, featuring flavors like Lavender Honey, Black Sesame Ginger, and Raspberry Rose. Designed to be savored slowly.","menu_name":"Parisian Dreamscape Collection"},{"menu_description":"A Parisian dream!  Five hand-painted bonbons, each like a miniature work of art, filled with unique and unexpected flavors like lavender honey, black truffle, and raspberry rose.","menu_name":"Parisian Bonbon Collection"},{"menu_description":"Five handcrafted dark chocolate squares, each infused with a different exotic spice - saffron, cardamom, ginger, black pepper, and star anise. Designed for sharing, this experience is a journey for the senses.","menu_name":"Aromatic Adventures"},{"menu_description":"Six miniature dark chocolate squares, each filled with a different exotic spice ganache, topped with a delicate edible gold leaf.","menu_name":"Spice Odyssey"},{"menu_description":"A playful twist on a classic!  Fresh, warm churros are coated in a cinnamon sugar blend  served with a rich, dark chocolate dipping sauce.  Topped with a scoop of vanilla bean ice cream and a sprinkle of sea salt.","menu_name":"Chocolate Churro Dreams"},{"menu_description":"A playful twist on a classic!  Three decadent chocolate spheres, each filled with a different mousse:  dark chocolate hazelnut, milk chocolate caramel, and white chocolate raspberry, served on a bed of crushed almond biscotti.","menu_name":"Parisian Chocolate Sphere Trio"},{"menu_description":"Five layers of our decadent chocolate cake are individually infused with a different exotic spice, then stacked high between layers of creamy dark chocolate ganache.  Finished with a drizzle of white chocolate.","menu_name":"Five Spice Chocolate Celebration Cake"},{"menu_description":"A delicate dark chocolate dome, infused with the rich aroma of freshly brewed coffee, and hiding a molten espresso heart. Topped with a delicate white chocolate swirl and a single coffee bean.","menu_name":"Molten Espresso Heart"},{"menu_description":"A single piece of our award winning Aztec Chocolate.  Made with Chili peppers grown in our own garden and high quality dark chocolate.","menu_name":"Aztec Spice"},{"menu_description":"A whimsical journey for two. This dessert features a dark chocolate globe, filled with layers of hazelnut dacquoise, espresso mousse, and a hint of orange. As warm chocolate sauce cascades down, the globe melts to reveal a hidden treasure of candied hazelnuts.","menu_name":"Melting Chocolate Globe Surprise"},{"menu_description":"A whimsical journey for two - miniature dark and white chocolate hot air balloons, filled with salted caramel mousse, floating above a bed of decadent chocolate soil.","menu_name":"Chocolate Hot Air Balloon Ride"},{"menu_description":"A whimsical dance of textures and temperatures! This dish features a warm chocolate lava cake with a molten dark chocolate center, crowned with a delicate spun sugar sphere dusted with edible gold. As you gently crack the sugar sphere, it reveals a scoop of vanilla bean ice cream nestled beside the cake, creating a delightful symphony of flavors.","menu_name":"Golden Orb Surprise"},{"menu_description":"A single-serving molten chocolate cake with a melted caramel center, served with a scoop of vanilla bean ice cream and chocolate shavings.","menu_name":"Molten Caramel Surprise"},{"menu_description":"A warm, rich chocolate lava cake with a molten center, served alongside a scoop of vanilla bean ice cream and drizzled with a salted caramel sauce.","menu_name":"Molten Chocolate Lava Cake"},{"menu_description":"A celebration of textures and temperatures, this dessert features a warm, gooey chocolate lava cake with a molten white chocolate center, paired with a scoop of refreshing lavender-infused ice cream and a sprinkle of crunchy cocoa nibs.","menu_name":"Lavender White Chocolate Lava Cake"},{"menu_description":"A decadent chocolate sphere filled with salted caramel, designed to melt at your table for a multisensory experience. As the sphere slowly melts, a cascade of delicate chocolate curls will be revealed, reminiscent of a blooming flower.","menu_name":"Blooming Caramel Truffle"},{"menu_description":"A whimsical assortment of hand-painted chocolates shaped as Parisian landmarks like the Eiffel Tower and the Louvre Pyramid. Flavors include raspberry ganache, salted caramel, and pistachio marzipan.","menu_name":"Parisian Dreamscape"},]
</ai-sweets-menu>

<ABCD-Rules>
Quick Pacing
Quick Pacing (First 5 seconds)
Dynamic Start
Supers
Supers with Audio
Brand Visuals
Brand Visuals (First 5 seconds)
Brand Mention (Speech)
Brand Mention (Speech) (First 5 seconds)
Product Visuals
Product Visuals (First 5 seconds)
Product Mention (Text)
Product Mention (Text) (First 5 seconds)
Product Mention (Speech)
Product Mention (Speech) (First 5 seconds)
Visible Face (First 5 seconds)
Visible Face (Close Up)
Presence of People
Presence of People (First 5 seconds)
Overall Pacing
Audio Speech Early
Call To Action (Text)
Call To Action (Speech)
</ABCD-Rules>


```

##### Gemini Sample Response

```
{
    "marketing-campaign-overview": "This video immerses viewers in the magic of Chocolate AI, showcasing signature offerings in a Parisian setting to ignite brand awareness & sweet cravings",
    "segments": [
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Quick Pacing (First 5 seconds)",
          "Dynamic Start",
          "Supers with Audio",
          "Brand Visuals",
          "Brand Visuals (First 5 seconds)",
          "Brand Mention (Speech)",
          "Brand Mention (Speech) (First 5 seconds)",
          "Visible Face (First 5 seconds)",
          "Visible Face (Close Up)",
          "Presence of People (First 5 seconds)",
          "Overall Pacing",
          "Audio Speech Early"
        ],
        "segement-number": 1,
        "segement-numbervisuals": 1,
        "text-to-video-prompt": "Camera pans around a bustling chocolate, dessert and coffee shop in Paris, showcasing delicious offerings with skilled chefs working. Charming and elegant, cinematic",
        "visuals": "Open on a close-up of a chocolatier's hands delicately piping intricate designs onto a Lavender Sea Salt Chocolate.  The camera slowly pans up to reveal their focused face, showcasing the passion in their craft.",
        "voiceover": "Welcome to Chocolate AI, where the art of chocolate making meets Parisian flair."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Text)",
          "Product Mention (Speech)"
        ],
        "segement-number": 2,
        "segement-numbervisuals": 2,
        "text-to-video-prompt": "Camera focuses on the creation of a decadent Spiced Chocolate Symphony cake with artistic flair.  A delicate hand places a single, hand-painted chocolate coffee bean atop the Spiced Chocolate Symphony cake in a chocolate, dessert, coffee shop in Paris. Charming and elegant, cinematic",
        "visuals": "A Spiced Chocolate Symphony cake is being assembled. Each layer carefully placed.  The camera zooms in on the chocolate coffee bean being delicately placed on top.",
        "voiceover": "Indulge in the unexpected with our Spiced Chocolate Symphony."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Speech)"
        ],
        "segement-number": 3,
        "segement-numbervisuals": 3,
        "text-to-video-prompt": "In a chocolate, dessert, coffee shop in Paris, a Parisian Chocolate Easter Egg is being delicately assembled.  The camera follows the delicate placement of truffles on a Parisian Chocolate Easter Egg. Charming and elegant, cinematic",
        "visuals": "Transition to a Parisian Chocolate Easter Egg being filled with an assortment of colorful truffles.",
        "voiceover": "Our Parisian Chocolate Easter Egg is a feast for the eyes and the palate."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Mention (Text)"
        ],
        "segement-number": 4,
        "segement-numbervisuals": 4,
        "text-to-video-prompt": "A Chocolate Espresso Cup is being made in a chocolate, dessert, coffee shop in Paris. The camera zooms in to show the making of a Chocolate Espresso Cup. Charming and elegant, cinematic",
        "visuals": "Cut to a shot of rich, dark Parisian Cafe Mocha being poured into a Chocolate Espresso Cup.",
        "voiceover": "Experience the perfect pairing; our Parisian Cafe Mocha and a Chocolate Espresso Cup."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Text)",
          "Product Mention (Speech)"
        ],
        "segement-number": 5,
        "segement-numbervisuals": 5,
        "text-to-video-prompt": "A Chocolate Caramel Orb is being made in a chocolate, dessert, coffee shop in Paris. Camera zooms out slowly on a Chocolate Caramel Orb in a chocolate, dessert, coffee shop in Paris. Charming and elegant, cinematic",
        "visuals": "The camera then focuses on a tray of glistening Chocolate Caramel Orbs.",
        "voiceover": "And for the ultimate indulgence, our Chocolate Caramel Orb."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Call To Action (Text)",
          "Call To Action (Speech)"
        ],
        "segement-number": 6,
        "segement-numbervisuals": 6,
        "text-to-video-prompt": "The storefront of Chocolate AI in Paris.  It is a beautiful day, with people walking past and the sun shining. Charming and elegant, cinematic",
        "visuals": "Transition to a shot of the exterior of Chocolate AI, bathed in the warm glow of the Parisian sun.",
        "voiceover": "Chocolate AI. Discover the extraordinary. Visit us in the heart of Paris."
      }
    ]
  }
```

##### Call Gemini to create a storyline for our Marketing Campaign

In [ ]:
def GetMenuItems():
  sql = f"""SELECT TO_JSON(STRUCT(menu_name, menu_description)) AS menu_item_json
  FROM `chocolate_ai.menu`"""

  result_df = RunQuery(sql)
  result_str = ""

  for index, row in result_df.iterrows():
    result_str = result_str + row['menu_item_json'] + ","

  return '[' + result_str + ']'

In [ ]:
# We need to tell the LLM how to write text-to-video prompts

text_to_video_prompt_guide = """
Text-to-Video Prompt Writing Help:
<text-to-video-prompt-guide>
Here are some our best practices for text-to-video prompts:

Detailed prompts = better videos:
  - More details you add, the more control you’ll have over the video. A prompt should look like this:
  - Example Prompt: "Camera dollies to show a close up of a desperate man in a green trench coat is making a call on a rotary style wall-phone, green neon light, movie scene."
    - Here is a break down of a text-to-video prompt:
    - Camera dollies to show = Camera motion
    - A close up of = Composition
    - A desperate man in a green trench coat = Subject
    - Is making a call = Action
    - On a roary style wall-phone = Scene
    - Green Neon light = Ambiance
    - Movie Scene = Style

Use the right keywords for better control:
  - We’ve identified a list of some keywords that work well with text-to-video, use these in your human written prompts to get the desired camera motion or style.
  - Subject: Who or what is the main focus of the shot e.g. happy woman in her 30s
  - Scene: Where is the location of the shot (on a busy street, in space)
  - Action: What is the subject doing (walking, running, turning head)
  - Camera Motion: What the camera is doing e.g. POV shot, Aerial View, Tracking Drone view, Tracking Shot
    - Example Camera Motion: "Tracking drone view of a man driving a red convertible car in Palm Springs, 1970s, warm sunlight, long shadows"
    - Example Camera Motion: "A POV shot from a vintage car driving in the rain, Canada at night, cinematic"


Styles:
   - Overall aesthetic. Consider using specific film style keywords e.g. horror film, film noir or animated styles e.g. 3D cartoon style render
  - Example Prompt: "Over the shoulder of a young woman in a car, 1970s, film grain, horror film, cinematic he Film noir style, man and woman walk on the street, mystery, cinematic, black and white"
  - Example Prompt: "A cute creatures with snow leopard-like fur is walking in winter forest, 3D cartoon style render. An architectural rendering of a white concrete apartment building with flowing organic shapes, seamlessly blending with lush greenery and futuristic elements."

Composition:
  - How the shot is framed. This is often relative to the subject e.g. wide shot, close-up, low angle
  - Example Prompt: "Extreme close-up of a an eye with city reflected in it. A wide shot of surfer walking on a beach with a surfboard, beautiful sunset, cinematic"

Ambiance & Emotions:
  - How the color and light contribute to the scene (blue tones, night)
  - Example Prompt: "A close-up of a girl holding adorable golden retriever puppy in the park, sunlight Cinematic close-up shot of a sad woman riding a bus in the rain, cool blue tones, sad mood"

Cinematic effects:
  - e.g. double exposure, projected, glitch camera effect
  - Example Prompt: "A double exposure of silhouetted profile of a woman walking and lake, walking in a forest Close-up shot of a model with blue light with geometric shapes projected on her face"
  - Example Prompt: "Silhouette of a man walking in collage of cityscapes Glitch camera effect, close up of woman’s face speaking, neon colors"
</text-to-video-prompt-guide>
"""

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "marketing-campaign-overview" : :"text",
#    "segments" : [
#                 {
#                   "segment-number" : 0,
#                   "visuals" : "",
#                   "voiceover" : "",
#                   "video-prompt" : "",
#                   "ABCD-rules" : ["",""],
#                 }
#                 ]
#  }
response_schema = {
  "type": "object",
  "required": [
    "marketing-campaign-overview",
    "segments"
  ],
  "properties": {
    "marketing-campaign-overview": {
      "type": "string"
    },
    "segments": {
      "type": "array",
      "items": {
        "type": "object",
        "required": [
          "segement-number",
          "segement-numbervisuals",
          "visuals",
          "voiceover",
          "text-to-video-prompt",
          "ABCD-rules"
        ],
        "properties": {
          "segement-number": {
            "type": "integer"
          },
          "segement-numbervisuals": {
            "type": "integer"
          },
          "visuals": {
            "type": "string"
          },
          "voiceover": {
            "type": "string"
          },
          "text-to-video-prompt": {
            "type": "string"
          },
          "ABCD-rules": {
            "type": "array",
            "items": {
              "type": "string"
            }
          }
        }
      }
    }
  }
}

menu_items = GetMenuItems()

gemini_ad_prompt = f"""You are a marketing expert and are creating a marketing video for a company that sells premium handcrafted chocolates, handmade desserts and delicious coffee drinks.
Write a {video_length_in_seconds}-second video script that showcases the artistry of chocolatiers crafting exquisite desserts.
This is a new company Chocolate AI who wants to build their brand awarness.
The company is based in Paris, France.
The video will be generated by GenAI and needs to be magicical and spark the imagination of the viewers.
Feature a range of signature offerings from the below menu.
Encourage unconventional ideas and fresh perspectives and inspires unique variations when viewing the video.
The video should be shot inside the cholcolate shop or show the street outside the shop.

Output Fields:
- "marketing-campaign-overview",
- "segments"
  - "segement-number": The segment number.
  - "segement-numbervisuals": The segment number.
  - "visuals": The visual screen that is being described.
  - "voiceover"  The script for the text-to-speech.  The number of words should be about 6 seconds when spoken.
  - "text-to-video-prompt":
    - Read the "Text-to-Video Prompt Writing Help" to learn more about how to create good text-to-video prompts.
    - Make sure you include all the relevant best practices when creating the text-to-video prompt:
    - A detailed prompt for generating the video using text-to-video technology.
    - Focus on creating the menu item with an artistic flair.
    - Do not include "text overlays" in the text-to-video prompt.
    - The prompt should also reference that we are in a chocolate, dessert, coffee shop in Paris so it knows the context of the video.
    - Do not include children in the text-to-video prompt.
  - "ABCD-rules": The relevant ABCD rules that should be applied to the segment.  The ABCD rules are a specific set of guidelines that you may or may not be using.

{text_to_video_prompt_guide}

<chocolate-ai-menu>
{menu_items}
</chocolate-ai-menu>

<ABCD-Rules>
Quick Pacing
Quick Pacing (First 5 seconds)
Dynamic Start
Supers
Supers with Audio
Brand Visuals
Brand Visuals (First 5 seconds)
Brand Mention (Speech)
Brand Mention (Speech) (First 5 seconds)
Product Visuals
Product Visuals (First 5 seconds)
Product Mention (Text)
Product Mention (Text) (First 5 seconds)
Product Mention (Speech)
Product Mention (Speech) (First 5 seconds)
Visible Face (First 5 seconds)
Visible Face (Close Up)
Presence of People
Presence of People (First 5 seconds)
Overall Pacing
Audio Speech Early
Call To Action (Text)
Call To Action (Speech)
</ABCD-Rules>
"""


multimodal_prompt_list = [
    { "text": gemini_ad_prompt }
    ]

print(gemini_ad_prompt)
llm_result = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)
gemini_ad_results_dict = json.loads(llm_result)
print()
print(gemini_ad_results_dict)

In [ ]:
print(PrettyPrintJson(llm_result))

### <font color='#4285f4'>Create local working directories</font>

In [ ]:
# Create a directory to download the videos from GCS and a directory to combine the text to speech and videos
directory_text_to_video = f"text-to-video-{formatted_date}"
directory_text_to_speech = f"text-to-speech-{formatted_date}"
directory_video_and_audio_en_GB = f"video-and-audio-en-GB{formatted_date}"
directory_video_and_audio_fr_FR = f"video-and-audio-fr-FR{formatted_date}"
directory_full_video = f"full-video-{formatted_date}"

# Holds the local video files (no audio)
os.makedirs(directory_text_to_video, exist_ok=True)
directory_text_to_video_path = os.getcwd() + f"/{directory_text_to_video}/"
print(f"directory_text_to_video_path: {directory_text_to_video_path}")

# Holds the audio file(s)
os.makedirs(directory_text_to_speech, exist_ok=True)
directory_text_to_speech_path = os.getcwd() + f"/{directory_text_to_speech}/"
print(f"directory_text_to_speech_path: {directory_text_to_speech_path}")

# Holds the video with audio (not merged seperate files) [en-GB]
os.makedirs(directory_video_and_audio_en_GB, exist_ok=True)
directory_video_and_audio_en_GB = os.getcwd() + f"/{directory_video_and_audio_en_GB}/"
print(f"directory_video_and_audio_en_GB: {directory_video_and_audio_en_GB}")

# Holds the video with audio (not merged seperate files) [fr-FR]
os.makedirs(directory_video_and_audio_fr_FR, exist_ok=True)
directory_video_and_audio_fr_FR = os.getcwd() + f"/{directory_video_and_audio_fr_FR}/"
print(f"directory_video_and_audio_fr_FR: {directory_video_and_audio_fr_FR}")

# Full video
os.makedirs(directory_full_video, exist_ok=True)
directory_full_video_path = os.getcwd() + f"/{directory_full_video}/"
print(f"directory_full_video_path: {directory_full_video_path}")

full_video_filename_with_audio_en_GB = directory_full_video_path + "full-video-with-audio-en-GB.mp4"
print(f"full_video_filename_with_audio_en_GB: {full_video_filename_with_audio_en_GB}")

full_video_filename_with_audio_fr_FR = directory_full_video_path + "full-video-with-audio-fr-FR.mp4"
print(f"full_video_filename_with_audio_fr_FR: {full_video_filename_with_audio_fr_FR}")

full_video_filename_no_audio = directory_full_video_path + "full-video-no-audio.mp4"
print(f"full_video_filename_no_audio: {full_video_filename_no_audio}")


### <font color='#4285f4'>Video Generation</font>

##### Veo 2

In [ ]:
def generateVideo(prompt, storage_account, output_gcs_path):
  """Calls text-to-video to create the video and waits for the output (which can be several minutes).  Saves the prompt/parameters with the vidoe.  Returns the outputted path."""

  full_output_gcs_path = f"gs://{storage_account}/{output_gcs_path}"
  model = "veo-2.0-generate-001"
  url = f"https://{location}-aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:predictLongRunning"

  request_body = {
      "instances": [
          {
              "prompt": prompt
          }
        ],
      "parameters": {
          "storageUri": full_output_gcs_path,
          "aspectRatio":"16:9"
          }
      }

  rest_api_parameters = request_body.copy()

  print(f"url: {url}")
  print(f"request_body: {request_body}")
  json_result = restAPIHelper(url, "POST", request_body)
  print(f"json_result: {json_result}")
  operation_name = json_result["name"] # odd this is name

  url = f"https://{location}-aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:fetchPredictOperation"

  request_body = {
      "operationName": operation_name
      }

  status = False

  while status == False:
    time.sleep(10)
    print(f"url: {url}")
    print(f"request_body: {request_body}")
    json_result = restAPIHelper(url, "POST", request_body)
    print(f"json_result: {json_result}")
    if "done" in json_result:
      status = bool(json_result["done"]) # in the future might be a status of running
    else:
      print("Status not present.  Assuming not done...")

  # Get the filename of our video
  filename = json_result["response"]["generatedSamples"][0]["video"]["uri"]

  # Save our prompt (this was we know what we used to generate the video)
  json_filename = "text-to-video-prompt.json"
  with open(json_filename, "w") as f:
    f.write(json.dumps(rest_api_parameters))

  # get the random number directory from text-to-video
  text_to_video_output_directory = filename.replace(full_output_gcs_path,"")
  text_to_video_output_directory = text_to_video_output_directory.split("/")[1]
  text_to_video_output_directory

  # Write the prompt to the same path as our outputted video.  Saving the prompt allow us to know how to regenerate it (you should also save the seed and any other settings)
  copy_file_to_gcs(json_filename, storage_account, f"{output_gcs_path}/{text_to_video_output_directory}/{json_filename}")
  delete_file(json_filename)

  return filename

##### Pre-Generated Video

In [ ]:
# Precanned Video #1
gemini_ad_results_dict = {
    "pre-generated-video" : True,
    "marketing-campaign-overview": "This video immerses viewers in the magic of Chocolate AI, showcasing signature offerings in a Parisian setting to ignite brand awareness & sweet cravings",
    "segments": [
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Quick Pacing (First 5 seconds)",
          "Dynamic Start",
          "Supers with Audio",
          "Brand Visuals",
          "Brand Visuals (First 5 seconds)",
          "Brand Mention (Speech)",
          "Brand Mention (Speech) (First 5 seconds)",
          "Visible Face (First 5 seconds)",
          "Visible Face (Close Up)",
          "Presence of People (First 5 seconds)",
          "Overall Pacing",
          "Audio Speech Early"
        ],
        "segement-number": 1,
        "segement-numbervisuals": 1,
        "text-to-video-prompt": "Camera pans around a bustling chocolate, dessert and coffee shop in Paris, showcasing delicious offerings with skilled chefs working. Charming and elegant, cinematic",
        "visuals": "Open on a close-up of a chocolatier's hands delicately piping intricate designs onto a Lavender Sea Salt Chocolate.  The camera slowly pans up to reveal their focused face, showcasing the passion in their craft.",
        "voiceover": "Welcome to Chocolate AI, where the art of chocolate making meets Parisian flair."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Text)",
          "Product Mention (Speech)"
        ],
        "segement-number": 2,
        "segement-numbervisuals": 2,
        "text-to-video-prompt": "Camera focuses on the creation of a decadent Spiced Chocolate Symphony cake with artistic flair.  A delicate hand places a single, hand-painted chocolate coffee bean atop the Spiced Chocolate Symphony cake in a chocolate, dessert, coffee shop in Paris. Charming and elegant, cinematic",
        "visuals": "A Spiced Chocolate Symphony cake is being assembled. Each layer carefully placed.  The camera zooms in on the chocolate coffee bean being delicately placed on top.",
        "voiceover": "Indulge in the unexpected with our Spiced Chocolate Symphony."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Speech)"
        ],
        "segement-number": 3,
        "segement-numbervisuals": 3,
        "text-to-video-prompt": "In a chocolate, dessert, coffee shop in Paris, a Parisian Chocolate Easter Egg is being delicately assembled.  The camera follows the delicate placement of truffles on a Parisian Chocolate Easter Egg. Charming and elegant, cinematic",
        "visuals": "Transition to a Parisian Chocolate Easter Egg being filled with an assortment of colorful truffles.",
        "voiceover": "Our Parisian Chocolate Easter Egg is a feast for the eyes and the palate."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Mention (Text)"
        ],
        "segement-number": 4,
        "segement-numbervisuals": 4,
        "text-to-video-prompt": "A Chocolate Espresso Cup is being made in a chocolate, dessert, coffee shop in Paris. The camera zooms in to show the making of a Chocolate Espresso Cup. Charming and elegant, cinematic",
        "visuals": "Cut to a shot of rich, dark Parisian Cafe Mocha being poured into a Chocolate Espresso Cup.",
        "voiceover": "Experience the perfect pairing; our Parisian Cafe Mocha and a Chocolate Espresso Cup."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Supers with Audio",
          "Product Visuals",
          "Product Mention (Text)",
          "Product Mention (Speech)"
        ],
        "segement-number": 5,
        "segement-numbervisuals": 5,
        "text-to-video-prompt": "A Chocolate Caramel Orb is being made in a chocolate, dessert, coffee shop in Paris. Camera zooms out slowly on a Chocolate Caramel Orb in a chocolate, dessert, coffee shop in Paris. Charming and elegant, cinematic",
        "visuals": "The camera then focuses on a tray of glistening Chocolate Caramel Orbs.",
        "voiceover": "And for the ultimate indulgence, our Chocolate Caramel Orb."
      },
      {
        "ABCD-rules": [
          "Quick Pacing",
          "Call To Action (Text)",
          "Call To Action (Speech)"
        ],
        "segement-number": 6,
        "segement-numbervisuals": 6,
        "text-to-video-prompt": "The storefront of Chocolate AI in Paris.  It is a beautiful day, with people walking past and the sun shining. Charming and elegant, cinematic",
        "visuals": "Transition to a shot of the exterior of Chocolate AI, bathed in the warm glow of the Parisian sun.",
        "voiceover": "Chocolate AI. Discover the extraordinary. Visit us in the heart of Paris."
      }
    ]
  }

##### Function generateVideo (Call Veo 2 REST API)

In [ ]:
def generateVideo(prompt, storage_account, output_gcs_path):
  """Calls text-to-video to create the video and waits for the output (which can be several minutes).  Saves the prompt/parameters with the vidoe.  Returns the outputted path."""

  full_output_gcs_path = f"gs://{storage_account}/{output_gcs_path}"
  model = "veo-2.0-generate-001"
  url = f"https://{location}-aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:predictLongRunning"

  request_body = {
      "instances": [
          {
              "prompt": prompt
          }
        ],
      "parameters": {
          "storageUri": full_output_gcs_path,
          "aspectRatio":"16:9"
          }
      }

  rest_api_parameters = request_body.copy()

  print(f"url: {url}")
  print(f"request_body: {request_body}")
  json_result = restAPIHelper(url, "POST", request_body)
  print(f"json_result: {json_result}")
  operation_name = json_result["name"] # odd this is name

  url = f"https://{location}-aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:fetchPredictOperation"

  request_body = {
      "operationName": operation_name
      }

  status = False
  # {
  # "name": "projects/chocolate-ai-demo-xxxxxx/locations/us-central1/publishers/google/models/veo-2.0-generate-001/operations/6d737b7c-5824-4f44-bc58-2e8d8226d2c2",
  # "done": True,
  # "response": {
  #      "@type": "type.googleapis.com/cloud.ai.large_models.vision.GenerateVideoResponse",
  #      "raiMediaFilteredCount": 0,
  #      "videos": [
  #          {
  #              "gcsUri": "gs: //chocolate-ai-data-xxxxxx/text-to-video/text-to-video-2025-04-15-13-59/9874965778463625250/sample_0.mp4",
  #              "mimeType": "video/mp4"
  #          }
  #      ]
  #  }
  # }

  while status == False:
    time.sleep(10)
    print(f"url: {url}")
    print(f"request_body: {request_body}")
    json_result = restAPIHelper(url, "POST", request_body)
    print(f"json_result: {json_result}")
    if "done" in json_result:
      status = bool(json_result["done"]) # in the future might be a status of running
    else:
      print("Status 'done' JSON attribute not present.  Assuming not done...")

  # Get the filename of our video
  filename = json_result["response"]["videos"][0]["gcsUri"]

  # Save our prompt (this was we know what we used to generate the video)
  json_filename = "text-to-video-prompt.json"
  with open(json_filename, "w") as f:
    f.write(json.dumps(rest_api_parameters))

  # get the random number directory from text-to-video
  text_to_video_output_directory = filename.replace(full_output_gcs_path,"")
  text_to_video_output_directory = text_to_video_output_directory.split("/")[1]
  text_to_video_output_directory

  # Write the prompt to the same path as our outputted video.  Saving the prompt allow us to know how to regenerate it (you should also save the seed and any other settings)
  copy_file_to_gcs(json_filename, storage_account, f"{output_gcs_path}/{text_to_video_output_directory}/{json_filename}")
  delete_file(json_filename)

  return filename

##### Create the Videos (using text-to-video)

In [ ]:
# Text-to-Video

for item in gemini_ad_results_dict["segments"]:
  filename = f"text-to-video-{item['segement-number']:02}.mp4"
  text_to_video_output_path_for_generated_video = f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}"

  if "pre-generated-video" in gemini_ad_results_dict:
    # Use pre-generated videos
    # Download the videos to the notebook computer
    download_from_gcs(f"{directory_text_to_video_path}{filename}", "data-analytics-golden-demo", f"chocolate-ai/v1/Campaign-Assets-Text-to-Video-02/story-01/{filename}")

    # Simulate that text-to-video generated the videos (pretend that we have the files outputed from text-to-video on storage)
    copy_file_to_gcs(f"{directory_text_to_video_path}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")
  else:
    # Generate videos
    generateVideo(item["text-to-video-prompt"], storage_account,"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")

    # Download the generate video from GCS
    download_from_gcs(f"{directory_text_to_video_path}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")

In [ ]:
# See the files
!ls {directory_text_to_video_path}

### <font color='#4285f4'>Text to Speech Generation</font>

In [ ]:
# Print out a list of language code, select one you want
response_text = TextToSpeechLanguageList("fr-FR")
response_json = json.loads(response_text)
print (response_json)
# print(PrettyPrintJson(response_text))

language_code_en_GB = "en-GB"
language_code_name_en_GB = "en-GB-Neural2-A"
ssml_gender_en_GB = "FEMALE"

language_code_fr_FR = "fr-FR"
language_code_name_fr_FR = "fr-FR-Neural2-A"
ssml_gender_fr_FR = "FEMALE"

In [ ]:
# Generate the text-to-speech for each segment
talking_speed = .85

for item in gemini_ad_results_dict["segments"]:
  # English
  filename = f"text-to-speech-{item['segement-number']:02}-en-GB.mp3"
  gcs_filename = f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}"

  TextToSpeech(f"{directory_text_to_speech_path}{filename}", item["voiceover"], language_code_en_GB, language_code_name_en_GB, ssml_gender_en_GB, talking_speed)
  display(Audio(f"{directory_text_to_speech_path}{filename}", autoplay=True,rate=16000))
  print()

  copy_file_to_gcs(f"{directory_text_to_speech_path}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")

  # French
  filename = f"text-to-speech-{item['segement-number']:02}-fr-FR.mp3"
  gcs_filename = f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}"

  TextToSpeech(f"{directory_text_to_speech_path}{filename}", item["voiceover"], language_code_fr_FR, language_code_name_fr_FR, ssml_gender_fr_FR, talking_speed)
  display(Audio(f"{directory_text_to_speech_path}{filename}", autoplay=True,rate=16000))
  print()

  copy_file_to_gcs(f"{directory_text_to_speech_path}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")


In [ ]:
!ls {directory_text_to_speech_path}

### <font color='#4285f4'>Combine Speech and Videos</font>

In [ ]:
# Merge the text-to-speech with the text-to-video

for item in gemini_ad_results_dict["segments"]:
  # English (source audio)
  source_audio_filename = f"text-to-speech-{item['segement-number']:02}-en-GB.mp3"
  source_vidio_filename = f"text-to-video-{item['segement-number']:02}.mp4"
  filename = f"text-to-video-{item['segement-number']:02}-en-GB.mp4"
  MergeVideoAndAudio(f"{directory_text_to_video_path}{source_vidio_filename}",
                     f"{directory_text_to_speech_path}{source_audio_filename}",
                     f"{directory_video_and_audio_en_GB}{filename}")

  copy_file_to_gcs(f"{directory_video_and_audio_en_GB}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")

  # French (source audio)
  source_audio_filename = f"text-to-speech-{item['segement-number']:02}-fr-FR.mp3"
  source_vidio_filename = f"text-to-video-{item['segement-number']:02}.mp4"
  filename = f"text-to-video-{item['segement-number']:02}-fr-FR.mp4"
  MergeVideoAndAudio(f"{directory_text_to_video_path}{source_vidio_filename}",
                     f"{directory_text_to_speech_path}{source_audio_filename}",
                     f"{directory_video_and_audio_fr_FR}{filename}")

  copy_file_to_gcs(f"{directory_video_and_audio_fr_FR}{filename}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/{filename}")


In [ ]:
# See the files
!ls {directory_video_and_audio_en_GB}

In [ ]:
# See the files
!ls {directory_video_and_audio_fr_FR}

In [ ]:
# Show the first merged file
filename = f"{directory_video_and_audio_en_GB}text-to-video-01-en-GB.mp4"
video_mp4 = open(filename, 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# Play the video
# 16:9 aspect ratio
HTML(f"""
<p>Combined the first segment video and audio (English)</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

In [ ]:
# Show the first merged file
filename = f"{directory_video_and_audio_fr_FR}text-to-video-01-fr-FR.mp4"
video_mp4 = open(filename, 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# Play the video
# 16:9 aspect ratio
HTML(f"""
<p>Combined the first segment video and audio (French)</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

### <font color='#4285f4'>Merge the videos into one video</font>

- **Warning: This does not currently produce a well edited video.**
- The code works, but a final version was created using a video editting software.

In [ ]:
# Merge the videos (the file names are when sorted match the placement of each video in the overall video)

print("Merging videos (English)")
merge_videos_sorted(directory_video_and_audio_en_GB, f"{full_video_filename_with_audio_en_GB}")

print("Merging videos (French)")
merge_videos_sorted(directory_video_and_audio_fr_FR, f"{full_video_filename_with_audio_fr_FR}")

In [ ]:
# Upload to GCS
copy_file_to_gcs(f"{full_video_filename_with_audio_en_GB}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/full-video-with-audio-en-GB.mp4")
copy_file_to_gcs(f"{full_video_filename_with_audio_fr_FR}", storage_account, f"chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/full-video-with-audio-fr-FR.mp4")

In [ ]:
# prompt: python to play a mp4 in a jupyter notebook
video_mp4 = open(f"{full_video_filename_with_audio_en_GB}", 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# 16:9 aspect ratio
HTML(f"""
<p>Combined video using moviepy which needs some fine tuning (English)</p>
<p>MoviePy when merging the audio needs some setting set to prevent audio issues.</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

In [ ]:
# prompt: python to play a mp4 in a jupyter notebook
video_mp4 = open(f"{full_video_filename_with_audio_fr_FR}", 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# 16:9 aspect ratio
HTML(f"""
<p>Combined video using moviepy which needs some fine tuning (French)</p>
<p>MoviePy when merging the audio needs some setting set to prevent audio issues.</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

### <font color='#4285f4'>View the Video in Cloud Storage</font>

In [ ]:
# To view the bucket
print(f"View the GCS bucket: https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}")

### <font color='#4285f4'>Ask Gemini what it thinks of the video</font>

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "video-analysis" : :"text",
#  }
response_schema = {
  "type": "object",
  "required": [
    "video-analysis"
  ],
  "properties": {
    "video-analysis": {
      "type": "string"
    }
  }
}


qa_prompt = """You are a video professional and need to critique a marketing video.
The video was generated by GenAI and you need to perform a quality check.
The video is for the company Chocolate AI based in Paris, France.
Does the video look real?
Are there anything things that look unrealistic about the video?
Take several passes in order to inspect each and every detail.
Think this through step by step.
"""


multimodal_prompt_list = [
    { "text": qa_prompt },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Text-to-Video-02/text-to-video-{formatted_date}/full-video-with-audio-en-GB.mp4" } },
    ]

print(qa_prompt)
llm_qa_result = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)
qa_results_dict = json.loads(llm_qa_result)
print()
print(PrettyPrintJson(llm_qa_result))

### <font color='#4285f4'>Clean Up</font>

In [ ]:
user_input = input("Do you want to the files on this notebook machine (Y/n)?")
if user_input == "Y":
  import shutil
  print(f"Removing directory: {directory_text_to_video_path}")
  shutil.rmtree(directory_text_to_video_path)

  print(f"Removing directory: {directory_text_to_speech_path}")
  shutil.rmtree(directory_text_to_speech_path)

  print(f"Removing directory: {directory_video_and_audio_en_GB}")
  shutil.rmtree(directory_video_and_audio_en_GB)

  print(f"Removing directory: {directory_video_and_audio_fr_FR}")
  shutil.rmtree(directory_video_and_audio_fr_FR)

  print(f"Removing directory: {directory_full_video_path}")
  shutil.rmtree(directory_full_video_path)

### <font color='#4285f4'>Reference Links</font>


- [Google.com](https://www.google.com)